In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.special import softmax

behaviors = ['Forage', 'Thermoregulate', 'Rest']
microhabitats = ['Burrow', 'Open']

static_utility_vector = np.array([
    [0, 1],  # Forage: Burrow = 0, Open = 1
    [1, 1],  # Thermoregulate: Burrow = 1, Open = 1
    [1, 0]   # Rest: Burrow = 1, Open = 0
])

In [16]:
print(enumerate(['Forage', 'Thermoregulate', 'Rest']))

In [4]:
# Constants
krat_cals_per_gram = 1.38
digestion_efficiency = 0.8
krat_mass = 70  # grams
krat_meal = krat_cals_per_gram * krat_mass * digestion_efficiency

max_meals = 10

max_thermal_quality = 5
max_metabolic_state = krat_meal*max_meals

thermal_quality = np.random.randint(0, 5)  # Random integer between 0 (inclusive) and 5 (exclusive)
metabolic_state = np.random.randint(0, 800)

# Scale utilities
metabolic_state_scaled = metabolic_state / max_metabolic_state
thermal_quality_scaled = thermal_quality / max_thermal_quality

# Compute utilities
U_forage = 1 - metabolic_state_scaled
U_rest = metabolic_state_scaled  # High preference for resting when energy is high
U_thermoregulate = thermal_quality_scaled  # High preference for thermoregulation when thermal need is high

preference_vector = np.array([[U_forage],
                             [U_thermoregulate],
                             [U_rest]])
print('Preference Vector')
print(f'Shape {preference_vector.shape}')
print(f'{preference_vector}')
print('_________________________')

print('Static Vector')
print(f'Shape {static_utility_vector.shape}')
print(f'{static_utility_vector}')
print('_________________________')

adjusted_utility = preference_vector*static_utility_vector

print('Adjusted Utility Vector')
print(f'Shape {adjusted_utility.shape}')
print(f'{adjusted_utility}')
masked_utility = np.where(adjusted_utility == 0, -np.inf, adjusted_utility)

softmax_probs = softmax(masked_utility.flatten())

print('Softmax Probability Distribution')
print(f'Shape {softmax_probs.shape}')
print(f'{softmax_probs}')

sampled_index = np.random.choice(len(softmax_probs), p=softmax_probs)

# Map the flat index back to the original 2D indices
rows, cols = adjusted_utility.shape
sampled_row, sampled_col = divmod(sampled_index, cols)

# Output results
print("Utilities:", masked_utility)
print("Flattened Probabilities:", softmax_probs)
print(f"Sampled Index (Flat): {sampled_index}")
print(f"Sampled Row, Col: ({behaviors[sampled_row]}, {microhabitats[sampled_col]})")


Preference Vector
Shape (3, 1)
[[0.02173913]
 [0.4       ]
 [0.97826087]]
_________________________
Static Vector
Shape (3, 2)
[[0 1]
 [1 1]
 [1 0]]
_________________________
Adjusted Utility Vector
Shape (3, 2)
[[0.         0.02173913]
 [0.4        0.4       ]
 [0.97826087 0.        ]]
Softmax Probability Distribution
Shape (6,)
[0.         0.15332449 0.22381445 0.22381445 0.39904661 0.        ]
Utilities: [[      -inf 0.02173913]
 [0.4        0.4       ]
 [0.97826087       -inf]]
Flattened Probabilities: [0.         0.15332449 0.22381445 0.22381445 0.39904661 0.        ]
Sampled Index (Flat): 4
Sampled Row, Col: (Rest, Burrow)


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import ipywidgets as widgets
from ipywidgets import interact

# Constants
krat_cals_per_gram = 1.38
digestion_efficiency = 0.8
krat_mass = 70  # grams
krat_meal = krat_cals_per_gram * krat_mass * digestion_efficiency

max_meals = 10
max_thermal_quality = 5
max_metabolic_state = krat_meal * max_meals

# Static utility vector (3x2 matrix)
static_utility_vector = np.array([
    [0, 1],  # Forage: Burrow, Open
    [1, 1],  # Thermoregulate: Burrow, Open
    [1, 0]   # Rest: Burrow, Open
])

# Define the function to generate and plot probabilities
def plot_probabilities(max_tq=5, max_ms=800, num_points=50):
    # Generate thermal quality and metabolic state ranges
    thermal_quality_values = np.linspace(0, max_tq, num_points)
    metabolic_state_values = np.linspace(0, max_ms, num_points)

    # Initialize arrays to store probabilities
    probabilities = []

    # Iterate over thermal quality and metabolic state combinations
    for tq in thermal_quality_values:
        row_probs = []
        for ms in metabolic_state_values:
            # Scale inputs
            metabolic_state_scaled = ms / max_metabolic_state
            thermal_quality_scaled = tq / max_thermal_quality

            # Compute preference vector
            U_forage = 1 - metabolic_state_scaled
            U_rest = metabolic_state_scaled
            U_thermoregulate = thermal_quality_scaled
            preference_vector = np.array([[U_forage],
                                           [U_thermoregulate],
                                           [U_rest]])

            # Compute adjusted utility
            adjusted_utility = preference_vector * static_utility_vector

            # Mask infeasible utilities
            masked_utility = np.where(adjusted_utility == 0, -np.inf, adjusted_utility)

            # Compute softmax probabilities
            softmax_probs = softmax(masked_utility.flatten())

            # Store total probabilities for the current combination
            row_probs.append(softmax_probs)

        probabilities.append(row_probs)

    # Convert probabilities to a NumPy array for easier plotting
    probabilities = np.array(probabilities)

    # Plot results
    fig, axs = plt.subplots(3, 2, figsize=(12, 12), sharex=True, sharey=True)

    behaviors = ['Forage', 'Thermoregulate', 'Rest']
    microhabitats = ['Burrow', 'Open']

    for i, behavior in enumerate(behaviors):
        for j, microhabitat in enumerate(microhabitats):
            c = axs[i, j].imshow(
                probabilities[:, :, i * 2 + j].T,  # Select the correct slice
                extent=[0, max_tq, 0, max_ms],
                origin='lower',
                aspect='auto',
                cmap='viridis',
                vmin=0,  # Set minimum value of color scale to 0
                vmax=1   # Set maximum value of color scale to 1
            )
            axs[i, j].set_title(f"{behavior} in {microhabitat}")
            axs[i, j].set_xlabel("Thermal Quality")
            axs[i, j].set_ylabel("Metabolic State")
            fig.colorbar(c, ax=axs[i, j], label='Probability')

    plt.tight_layout()
    plt.show()

# Create sliders for interactive input
interact(
    plot_probabilities,
    max_tq=widgets.FloatSlider(value=5, min=1, max=10, step=0.5, description='Max Thermal Quality'),
    max_ms=widgets.FloatSlider(value=800, min=200, max=1500, step=50, description='Max Metabolic State'),
    num_points=widgets.IntSlider(value=50, min=10, max=100, step=10, description='Resolution')
)


interactive(children=(FloatSlider(value=5.0, description='Max Thermal Quality', max=10.0, min=1.0, step=0.5), …

<function __main__.plot_probabilities(max_tq=5, max_ms=800, num_points=50)>

In [2]:
import numpy as np
print(np.zeros(3, dtype=float))

[0. 0. 0.]
